In [2]:
# Import des packages nécessaires
# Import bibliothèque de manipulation de dataframe
import pandas as pd

# Import des bibliothèques de viz
import matplotlib.pyplot as plt
import seaborn as sns

# Import split data
from sklearn.model_selection import train_test_split

# Import modèles de ML Supervisé Régression
from sklearn.linear_model import LinearRegression

# Import modèles de ML Supervisé Classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Import modèle de ML NON Supervisé
from sklearn.neighbors import NearestNeighbors

# Import des métriques
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

# Import outil standardisation de la donnée
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer

# Import pipeline
from sklearn.pipeline import Pipeline

# Gestion des warnings
import warnings

In [3]:
df = pd.read_csv('../ressources/df_final.csv', sep=';', index_col=0)

In [4]:
df

,primaryTitle,title,frenchTitle,startYear,genres,production_countries,runtimeMinutes,revenue,tagline,overview,id,averageRating,numVotes,actor,actress,director,producer,writer
tconst,,,,,,,,,,,,,,,,,,
tt0010307,J'accuse!,J'accuse,J'accuse,1919,"Horror, War, Romance, Drama, History",FR,166,0.0,NaN,"The story of two men, one married, the other t...",70804,7.7,2240,"Blaise Cendrars, Maxime Desjardins, Paul Duc, ...","Angèle Guys, Elizabeth Nizan, Mancini, Maryse ...",Abel Gance,Charles Pathé,Abel Gance
tt0013933,The Faithful Heart,Cœur fidèle,Coeur fidèle,1923,"Romance, Drama",FR,87,0.0,NaN,The good guys win out in this sweet tale about...,67440,7.4,1540,"Claude Benedict, Edmond Van Daële, Léon Mathot","Gina Manès, Madame Maufroy, Madeleine Erickson...",Jean Epstein,NaN,"Jean Epstein, Marie Epstein"
tt0015014,L'inhumaine,L'Inhumaine,L'inhumaine,1924,"Romance, ScienceFiction, Drama, Mystery",FR,135,0.0,NaN,"A famous singer Claire Lescot, who lives on th...",54539,7.2,1134,"Fred Kellerman, Jaque Catelain, Las Bonambella...","Georgette Leblanc, Kiki of Montparnasse, Marce...",Marcel L'Herbier,Marie-Laure de Noailles,"Georgette Leblanc, Joris-Karl Huysmans, Marcel..."
tt0017075,The Lodger: A Story of the London Fog,The Lodger: A Story of the London Fog,Les cheveux d'or,1927,"Thriller, Drama, Crime, Mystery",GB,92,0.0,NaN,London. A mysterious serial killer brutally mu...,2760,7.3,14289,"Arthur Chesney, Ivor Novello, Malcolm Keen, Re...","Daisy Campbell, Eve Gray, June Tripp, Marie Au...",Alfred Hitchcock,NaN,"Eliot Stannard, Marie Belloc Lowndes"
tt0017196,Nana,Nana,Nana,1926,"Romance, Drama",FR,150,0.0,NaN,"A government official, Count Muffat, falls und...",66812,6.6,1011,"Claude Autant-Lara, Jean Angelo, Karl Harbache...","Catherine Hessling, Jacqueline Forzane, Valesk...",Jean Renoir,Jean Renoir,"Denise Leblond, Pierre Lestringuez, Émile Zola"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9902160,Herself,Herself,Herself,2020,Drama,"IE,GB",97,0.0,Life under construction,Struggling to provide her daughters with a saf...,653708,7.0,5099,"Art Kearns, Ian Lloyd Anderson, Shadaan Felfeli","Anita Petry, Cathy Belton, Clare Dunne, Ericka...",Phyllida Lloyd,"Ed Guiney, Rory Gilmartin, Sharon Horgan","Clare Dunne, Malcolm Campbell"
tt9904802,Enemy Lines,Enemy Lines,Enemy Lines,2020,"Drama, Action, War",GB,92,0.0,Courage has no border.,"In the frozen, war torn landscape of occupied ...",679796,4.6,2041,"Corey Johnson, Daniel Jillings, Ed Westwick, G...",Maria Bondareva,Anders Banke,"Aleksandr Kushaev, Andy Thompson, Nadzeya Huse...","Michael Wright, Tom George"
tt9908390,Le lion,The Lion,Le lion,2020,Comedy,FR,95,0.0,NaN,A psychiatric hospital patient pretends to be ...,589970,5.5,1497,"Aksel Ustun, Benoît Pétré, Dany Boon, Mathieu ...","Anne Serra, Carole Brana, Sophie Verbeeck",Ludovic Colbeau-Justin,"Jean-Yves Robin, Marc Stanimirovic, Serge Hayat","Alexandre Coquelle, Matthieu Le Naour"


In [12]:
"""df_genre = (
    df
    .assign(genres = df['genres'].str.split(','))  # chaînage en liste
    .explode('genres')                                  # une ligne par genre
    .assign(genres = lambda d: d['genres'].str.strip()) # suppression des espaces en trop
    .query("genres != '' and genres == genres")
    )

df_genre = df_genre.groupby('genres').agg(
    averageRating=('averageRating', 'mean'),
    count=('averageRating', 'size')
    ).reset_index()

df_genre"""

# Transformation de la colonne 'genres' en liste
genres_list = df['genres'].str.split(',').apply(lambda lst: [g.strip() for g in lst])

# Appliquer le MultiLabelBinarizer pour transformer la colonne 'genres' en colonnes binaires
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(genres_list)

# Créer un DataFrame avec les colonnes binaires
genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_)

genres_df


,,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,Mystery,Romance,Sci-Fi,ScienceFiction,Sport,TVMovie,Thriller,Unknown,War,Western
0,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,1,1,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,1,0,...,1,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12660,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12661,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
12662,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12663,0,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [13]:
# Concatener le DataFrame original avec le DataFrame des genres binaires
df_with_genres = pd.concat([df, genres_df], axis=1)

df_with_genres = df_with_genres.drop(columns=['genres'], axis=1)

display(df_with_genres.columns)

Index(['tconst', 'primaryTitle', 'title', 'frenchTitle', 'startYear',
       'production_countries', 'runtimeMinutes', 'revenue', 'tagline',
       'overview', 'id', 'averageRating', 'numVotes', 'actor', 'actress',
       'director', 'producer', 'writer', '', 'Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'ScienceFiction', 'Sport',
       'TVMovie', 'Thriller', 'Unknown', 'War', 'Western'],
      dtype='object')

In [6]:
production_countries_exploded = (
    df["production_countries"]
    .str.split(",")
    .explode()
    .str.strip()
)
print(len(production_countries_exploded.unique()))
production_countries_exploded.unique()

168


array(['FR', 'GB', 'DE', 'IN', 'SE', 'US', 'TR', 'IT', 'ES', 'CH', 'MX',
       'JP', 'YU', 'BR', 'LI', 'PL', 'AT', 'GR', 'MC', 'ZA', 'RO', 'SN',
       'AU', 'BE', 'CA', 'DZ', 'IE', 'XC', 'LU', 'RU', 'SU', 'TN', 'DK',
       'IR', 'MA', 'KW', 'LB', 'LY', 'SA', 'CI', 'VE', 'AR', 'NL', 'HK',
       'HU', 'PT', 'IL', 'PS', 'NZ', 'MQ', 'CO', 'ZW', 'CN', 'BF', 'ML',
       'CM', 'CL', 'FI', 'CU', 'NI', 'VN', 'NO', 'PG', 'NA', 'IS', 'SK',
       'GW', 'PA', 'HT', 'TW', 'BY', 'KH', 'CZ', 'ET', 'GH', 'GN', 'UA',
       'UY', 'GA', 'EG', 'MK', 'EE', 'BG', 'AL', 'BA', 'GE', 'SI', 'LT',
       'PR', 'RS', 'ID', 'MY', 'SG', 'KE', 'CY', 'AN', 'TJ', 'LV', 'MR',
       'KG', 'CD', 'UZ', 'KR', 'PM', 'NP', 'TH', 'TD', 'TT', 'CG', 'KZ',
       'PE', 'MT', 'AI', 'CF', 'PK', 'BD', 'BT', 'AM', 'IQ', 'JM', 'MU',
       'LK', 'SZ', 'AF', 'DO', 'GP', 'HR', 'YE', 'PY', 'QA', 'KY', 'NG',
       'DJ', 'BJ', 'MN', 'AE', 'CR', 'SY', 'LR', 'GT', 'BO', 'XK', 'JO',
       'TZ', 'RW', 'NC', 'PH', 'XI', 'PF', 'AZ', 'M

In [7]:
# Préparation KNN non supervisé
# Definir les ensembles de features pour trouvers les voisins similaires
# Definir le nombre de voisins
# Definir le modèle KNN


# Définir X_class qui contient les features pour la classification qui contient les variables : 
# frenchTitle, startYear, genres, production_countries, revenue, averageRating, numVotes, actor, actress, director, producer, writer.

df.reset_index(inplace=True)
X_class = df.drop(columns=['tconst', 'title', 'primaryTitle', 'revenue', 'runtimeMinutes', 'tagline', 'overview', 'id', 'numVotes'])

X_class


,frenchTitle,startYear,genres,production_countries,averageRating,actor,actress,director,producer,writer
0,J'accuse,1919,"Horror, War, Romance, Drama, History",FR,7.7,"Blaise Cendrars, Maxime Desjardins, Paul Duc, ...","Angèle Guys, Elizabeth Nizan, Mancini, Maryse ...",Abel Gance,Charles Pathé,Abel Gance
1,Coeur fidèle,1923,"Romance, Drama",FR,7.4,"Claude Benedict, Edmond Van Daële, Léon Mathot","Gina Manès, Madame Maufroy, Madeleine Erickson...",Jean Epstein,NaN,"Jean Epstein, Marie Epstein"
2,L'inhumaine,1924,"Romance, ScienceFiction, Drama, Mystery",FR,7.2,"Fred Kellerman, Jaque Catelain, Las Bonambella...","Georgette Leblanc, Kiki of Montparnasse, Marce...",Marcel L'Herbier,Marie-Laure de Noailles,"Georgette Leblanc, Joris-Karl Huysmans, Marcel..."
3,Les cheveux d'or,1927,"Thriller, Drama, Crime, Mystery",GB,7.3,"Arthur Chesney, Ivor Novello, Malcolm Keen, Re...","Daisy Campbell, Eve Gray, June Tripp, Marie Au...",Alfred Hitchcock,NaN,"Eliot Stannard, Marie Belloc Lowndes"
4,Nana,1926,"Romance, Drama",FR,6.6,"Claude Autant-Lara, Jean Angelo, Karl Harbache...","Catherine Hessling, Jacqueline Forzane, Valesk...",Jean Renoir,Jean Renoir,"Denise Leblond, Pierre Lestringuez, Émile Zola"
...,...,...,...,...,...,...,...,...,...,...
12660,Herself,2020,Drama,"IE,GB",7.0,"Art Kearns, Ian Lloyd Anderson, Shadaan Felfeli","Anita Petry, Cathy Belton, Clare Dunne, Ericka...",Phyllida Lloyd,"Ed Guiney, Rory Gilmartin, Sharon Horgan","Clare Dunne, Malcolm Campbell"
12661,Enemy Lines,2020,"Drama, Action, War",GB,4.6,"Corey Johnson, Daniel Jillings, Ed Westwick, G...",Maria Bondareva,Anders Banke,"Aleksandr Kushaev, Andy Thompson, Nadzeya Huse...","Michael Wright, Tom George"
12662,Le lion,2020,Comedy,FR,5.5,"Aksel Ustun, Benoît Pétré, Dany Boon, Mathieu ...","Anne Serra, Carole Brana, Sophie Verbeeck",Ludovic Colbeau-Justin,"Jean-Yves Robin, Marc Stanimirovic, Serge Hayat","Alexandre Coquelle, Matthieu Le Naour"
12663,Safeguard,2020,"Thriller, Adventure, Action, Crime",GB,3.6,"Akie Kotabe, Andrei Nova, Lee Byford, Leo Ashi...",Yuriri Naka,Fraser Precious,"Fraser Precious, Megan Young",Fraser Precious


In [8]:
# Adapter le dataframe pour n'avoir ques des valeurs numériques


In [9]:
# Entrainer le modèle KNN

In [10]:
# Prédire quelques voisins
# Afficher les voisins